In [ ]:

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, KFold, cross_val_predict

# from sklearn.naive_bayes import GaussianNB accuracy = 50%



with open('pss.txt') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
password = lines[0]
db = lines[1]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database=db,
    use_pure=True
)

my_cursor = mydb.cursor()
steps_title_list = []
d_steps_title = {}
university_name_list = []
d_uni_name = {}

def convert_nans(df):
    # job_applicant_id,gender,age,marriage_status,language,degree,skill,num_prev_company,work_interval,steps_title
    df['steps_title'] = df['steps_title'].fillna(6)
    df['age'] = df['age'].fillna(-1)
    df['skill'] = df['skill'].fillna(-1)
    df['marriage_status'] = df['marriage_status'].fillna(-1)
    df['language'] = df['language'].fillna(-1)
    # df['job_contract_type'] = df['job_contract_type'].fillna(-1)
    df['gender'] = df['gender'].fillna(-1)
    df['degree'] = df['degree'].fillna(-1)
    df['skill'] = df['skill'].fillna(-1)
    df['num_prev_company'] = df['num_prev_company'].fillna(-1)
    df['work_interval'] = df['work_interval'].fillna(-1)





my_cursor.execute('select distinct steps_title from useful_data')
for item in my_cursor:
    steps_title_list.append(item)


for title in steps_title_list:
    if title[0] == 'نیازمند تعیین وضعیت':
        d_steps_title[title[0]] = 0
    elif title[0] == 'تایید برای مصاحبه':
        d_steps_title[title[0]] = 1
    elif title[0] == 'استخدام شده':
        d_steps_title[title[0]] = 2
    elif title[0] == 'رد شده':
        d_steps_title[title[0]] = 3
    elif title[0] == 'انصراف از مصاحبه':
        d_steps_title[title[0]] = 4
    else:
        d_steps_title[title[0]] = 6


df_tagged = pd.read_csv("tagged_data.csv")
df_tagged['steps_title'] = df_tagged['steps_title'].map(d_steps_title)
convert_nans(df_tagged)
features_tagged = df_tagged.columns[:9]
x_train , x_test , y_train , y_test = train_test_split(df_tagged[features_tagged] , df_tagged['steps_title'], test_size=0.2 , random_state=42)
kfold = KFold(n_splits=10)
clf = RandomForestClassifier(n_jobs=2 , random_state=0)
cv_results = cross_val_score(clf,x_train , y_train,cv=kfold , scoring='accuracy')
y_pred=cross_val_predict(clf,x_train,y_train,cv=10)
results = (cv_results.mean(),cv_results.std())
print(results)
print("accuracy : ", accuracy_score(y_train, y_pred))
pd.crosstab(y_train,y_pred)